<h1>Data Cleaning</h1>

<p>This is the second step of the NLP pipeline. For details on the first step (data collection), look under "scrapers/" to find the script that scrapes Reuters for company news data based on the stock ticker.</p>
<p>The columns that have been scraped are</p>
<ul>
    <li>Date</li>
    <li>Headline</li>
    <li>First sentence of the article</li>
</ul>